In [1]:
from glob import glob

files = glob('*.parse')
files

['mnli.tsv.parse',
 'dumping-iium.tsv.parse',
 'dumping-parliament.tsv.parse',
 'qa.tsv.parse',
 'chatbot.tsv.parse',
 'dumping-wiki.tsv.parse',
 'dumping-news.tsv.parse',
 'dumping-pdf.tsv.parse',
 'summary.tsv.parse',
 'quora.tsv.parse',
 'snli.tsv.parse',
 'dumping-watpadd.tsv.parse',
 'stemming.tsv.parse',
 'news-title.tsv.parse',
 'synonym.tsv.parse']

In [2]:
from unidecode import unidecode
from tqdm import tqdm
import collections
import tensorflow as tf
maxlen = 1024

def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list = tf.train.Int64List(value = list(values))
    )
    return feature


In [3]:
def get_inputs(x, filename):
    input_ids, input_masks, segment_ids, ys = [], [], [], []
    for i in tqdm(range(len(x))):
        tokens = x[i][0]
        input_id = tokens
        
        segment_id = [0] * len(tokens)
        input_mask = [1] * len(input_id)
        
        input_id = input_id + [0] * (maxlen - len(input_id))
        segment_id = segment_id + [0] * (maxlen - len(segment_id))
        input_mask = input_mask + [0] * (maxlen - len(input_mask))

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        
        r = x[i][1]
        r = r + [0] * (maxlen - len(r))
        ys.append(r)
        
    r = tf.python_io.TFRecordWriter(f'{filename}.tfrecord')
    for i in tqdm(range(len(ys))):
        features = collections.OrderedDict()
        features['input_ids'] = create_int_feature(input_ids[i])
        features['input_mask'] = create_int_feature(input_masks[i])
        features['segment_ids'] = create_int_feature(segment_ids[i])
        features['y'] = create_int_feature(ys[i])
        tf_example = tf.train.Example(
            features = tf.train.Features(feature = features)
        )
        r.write(tf_example.SerializeToString())
    r.close()

In [4]:
import json

for file in files:
    print(file)

    with open(file) as fopen:
        data = json.load(fopen)
    
    get_inputs(data, file)

mnli.tsv.parse


100%|██████████| 387577/387577 [05:47<00:00, 1116.93it/s]


dumping-iium.tsv.parse


100%|██████████| 19477/19477 [00:17<00:00, 1090.20it/s]


dumping-parliament.tsv.parse


...

100%|██████████| 200000/200000 [02:59<00:00, 1111.57it/s]


news-title.tsv.parse


100%|██████████| 120410/120410 [01:47<00:00, 1117.19it/s]


synonym.tsv.parse


100%|██████████| 150000/150000 [02:17<00:00, 1091.58it/s]


In [2]:
files = glob('*.tfrecord')
files

['dumping-parliament.tsv.parse.tfrecord',
 'mnli.tsv.parse.tfrecord',
 'dumping-iium.tsv.parse.tfrecord',
 'qa.tsv.parse.tfrecord',
 'stemming.tsv.parse.tfrecord',
 'dumping-pdf.tsv.parse.tfrecord',
 'chatbot.tsv.parse.tfrecord',
 'synonym.tsv.parse.tfrecord',
 'dumping-watpadd.tsv.parse.tfrecord',
 'news-title.tsv.parse.tfrecord',
 'dumping-wiki.tsv.parse.tfrecord',
 'quora.tsv.parse.tfrecord',
 'summary.tsv.parse.tfrecord',
 'dumping-news.tsv.parse.tfrecord',
 'snli.tsv.parse.tfrecord']

In [3]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-storage.json'

In [4]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [5]:
for file in files:
    print(file)
    blob = bucket.blob(f'b2b-data/{file}')
    blob.upload_from_filename(file)

dumping-parliament.tsv.parse.tfrecord
mnli.tsv.parse.tfrecord
dumping-iium.tsv.parse.tfrecord
qa.tsv.parse.tfrecord
stemming.tsv.parse.tfrecord
dumping-pdf.tsv.parse.tfrecord
chatbot.tsv.parse.tfrecord
synonym.tsv.parse.tfrecord
dumping-watpadd.tsv.parse.tfrecord
news-title.tsv.parse.tfrecord
dumping-wiki.tsv.parse.tfrecord
quora.tsv.parse.tfrecord
summary.tsv.parse.tfrecord
dumping-news.tsv.parse.tfrecord
snli.tsv.parse.tfrecord


In [7]:
# !pip3 freeze